Video of Selected VMGD WebCam Images
-- 

Assumes all images have the same dimensions as the first image in the folder, generally no reason this should not be the case

In [15]:
import cv2
import os
import datetime
from datetime import timedelta
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout

In [16]:
def imfdt(imfile):
    #gets a datetime for an image file named like LVVL20180101T0000VUT.jpg or AMB120180430T0845VUT.jpg
    
    #drop site name at start of imfile name
    imfile = imfile[len(site):]
    
    #extract digits from string, these are the date-time
    imdt = ''.join(list(filter(str.isdigit, imfile)))
    year = imdt[:4]
    month = imdt[4:6]
    day = imdt[6:8]
    hour = imdt[8:10]
    minute = imdt[10:12]
    second = 0
    dt = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), second)
    return (dt)

In [17]:
def selsite(s):
    global site, volcano
    sitvol = {'AMB1':'Ambrym', 'LVVL':'Ambae', 'LPV':'Lopevi', 'YASH':'Tanna'}
    site = s['new']
    volcano = sitvol.get(site)
    print (site, volcano)
    
def seldate1(d1):
    global date1, time_start
    date = d1['new']
    date1 = date.strftime("%Y-%m-%d")
    time_start = datetime.datetime.strptime(date1, '%Y-%m-%d')
    print (date1, time_start)

def seldate2(d2):
    global date2, time_end
    date = d2['new']
    date2 = date.strftime("%Y-%m-%d")
    time_end = datetime.datetime.strptime(date2, '%Y-%m-%d')
    print (date2, time_end)

def makevideo(m):
    print ('making video')
      
    #video path and file name
    vidname = site+'_'+date1+'_'+date2+'_video.avi'
    vidfile = os.path.join(viddir, vidname)
    
    #unique list of years/months we need to examine
    years = []
    months = []
    for date in pd.date_range(date1, date2):
        years.append(date.year)
        months.append(date.month)
    #unique values, no better way of doing this?
    myset = set(years)
    years = list(myset)
    myset = set(months)
    months = list(myset)
    print (years, months)
    
    #get size of last image, required for video specification
#     im1 = sorted(os.listdir(os.path.join(imgdir,volcano,str(years[0]),str(months[0]).zfill(2))))[0]
#     img1 = os.path.join(imgdir,volcano,str(years[0]),str(months[0]).zfill(2),im1)
    im1 = sorted(os.listdir(os.path.join(imgdir,volcano)))[-1]
    img1 = os.path.join(imgdir,volcano,im1)
    
    img = cv2.imread(img1)
    height,width,layers = img.shape
    #video specifications
    fps = 4
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video = cv2.VideoWriter(vidfile,fourcc,fps,(width,height))
    
    #loop required years and months, then examine files to see if qualify    
    for year in years:
        #print(year)
        for month in months:
            #print (month)
            searchdir = os.path.join(imgdir,volcano,str(year),str(month).zfill(2))
            if os.path.isdir(searchdir):
                for file in sorted(os.listdir(searchdir)):
                    if file.endswith('.jpg'):
                        imtime = imfdt(file)
                        if (imtime > time_start and imtime < time_end):
                            imfile = os.path.join(searchdir, file)
                            img = cv2.imread(imfile)
                            img[y_offset:y_offset+logo.shape[0], x_offset:x_offset+logo.shape[1]] = logo
                            video.write(img)
                        
    #most recent files in year (not yet in month folder), so examine those too
    for year in years:
        #print(year)
        searchdir = os.path.join(imgdir,volcano)
        for file in sorted(os.listdir(searchdir)):
            if file.endswith('.jpg'):
                imtime = imfdt(file)
                if (imtime > time_start and imtime < time_end):
                    imfile = os.path.join(searchdir, file)
                    img = cv2.imread(imfile)
                    img[y_offset:y_offset+logo.shape[0], x_offset:x_offset+logo.shape[1]] = logo
                    video.write(img)
    
    cv2.destroyAllWindows
    video.release()
    print ('**VIDEO COMPLETE**')

In [18]:
imgdir = '/home/sherburn/BenbowWebcam'
viddir = '/home/sherburn/scratch/vanuatu_video'

#positional offset for logo
x_offset=5
y_offset=400

#read in logo
logo = cv2.imread('/home/sherburn/Dropbox/work/Vanuatu_ipynb/video_logo.png')

**Set directories**

In [19]:
s = widgets.Select(
    options=['select site', 'AMB1', 'LPV', 'LVVL', 'YASH'],
    value='select site',
    description='Site:',
    disabled=False
)
s.observe(selsite, names='value')

d1 = widgets.DatePicker(
  description='Start Date:')
d1.observe(seldate1, names='value')

d2 = widgets.DatePicker(
  description='End Date:')
d2.observe(seldate2, names='value')

m=widgets.Button(
    description='Make video',
    disabled=False,
    button_style='success',
    tooltip='Click to begin making video')
m.on_click(makevideo)

widgets.HBox([s, d1, d2, m])

AMB1 Ambrym
2018-03-08 2018-03-08 00:00:00
2018-03-10 2018-03-10 00:00:00
making video
[2018] [3]
**VIDEO COMPLETE**
